In [ ]:
import sqlite3
from wsgiref.simple_server import make_server
import tkinter

def get_form_vals(post_str):
	form_vals = {item.split("=")[0]: item.split("=")[1] for item in post_str.decode().split("&")}
	return form_vals
def countries_app(environ, start_response):
	conn = sqlite3.connect("final_exam.sqlite")
	cursor = conn.cursor()
	message = "<h1>COUNTRIES DATA</h1>"
	if(environ['REQUEST_METHOD'] == 'POST'):
		request_body_size = int(environ['CONTENT_LENGTH'])
		request_body = environ['wsgi.input'].read(request_body_size)
		form_vals = get_form_vals(request_body)
		try:
			sql="INSERT INTO COUNTRIES(Country, TotalArea) Values('"+form_vals['country']+"', '"+form_vals['area']+"')"
			cursor.execute(sql)
			conn.commit()
		except:
			message+="<font color=red>an error has occurred</font>"
	status = '200 OK'
	headers = [('Content-type', 'html; charset=utf-8')]
	start_response(status, headers)
	result = cursor.execute("select Country, TotalArea from COUNTRIES")
	message+="<table>"
	message+="<tr><td>Country</td><td>Total Area</td></tr>"
	for row in result:
		message+="<tr>"
		for column in row:
			message+="<td>"+str(column)+"</td>"
		message+="</tr>"
	message+="</table><br/>"
	message+="<form method='POST'>Add Record:<br/><br/>"
	message+="Country:<input type='text' name='country'><br/><br/>"
	message+="Total Area:<input type='number' name='area'><br/><br/>"
	message+="<input type='Submit'>"
	return[bytes(message,'utf-8')]
	conn.close()
	

'''
base = tkinter.Tk()
base.title("Countries Data")
canvas = tkinter.Canvas(base, bg="white", height=1000, width=1000)
conn = sqlite3.connect("Final_Exam.sqlite")
cursor = conn.cursor()
result = cursor.execute("select Country, TotalArea from COUNTRIES")
l = 0
for row in result:
	l+=50
	canvas.create_text(100,l-10, text=row[0])
	canvas.create_line(200,l , int(row[1])/50000+200, l, fill="black")
canvas.pack()
base.mainloop()
'''
httpd = make_server('', 8000, countries_app)
print("Serving on port 8000...")


httpd.serve_forever()